In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

/bin/bash: google-drive-ocamlfuse: command not found
drive  sample_data


In [2]:
import os
os.chdir("/content/drive/My Drive/yüz tanıma/")


In [3]:
!ls

data  model1.h5  yüz_tanıma.ipynb


In [6]:
!pip install -q keras

In [7]:
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.3


In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, save_model
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
import numpy as np

In [31]:
train_data_gen = ImageDataGenerator(rescale=1/255.,
                                    rotation_range=40,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode="nearest")

val_data_gen=ImageDataGenerator(rescale=1/255.)

train_data=train_data_gen.flow_from_directory(directory="/content/drive/My Drive/yüz tanıma/data/train",target_size=(224,224),color_mode="grayscale",class_mode="categorical", batch_size=16)

val_data=val_data_gen.flow_from_directory(directory="/content/drive/My Drive/yüz tanıma/data/validation",target_size=(224,224),color_mode="grayscale",class_mode="categorical", batch_size=16)

print(type(train_data))


Found 303 images belonging to 3 classes.
Found 75 images belonging to 3 classes.
<class 'tensorflow.python.keras.preprocessing.image.DirectoryIterator'>


In [33]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(12,12))
for i in range(0,15):
  plt.subplot(5,3,i+1)
  for X_batch,Y_batch in train_data:
    image=X_batch[i]
    
  plt.imshow(image)
plt.tight_layout()
plt.show()

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(32,(3,3),input_shape=(224,224,1),activation="relu"))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(3, activation="softmax"))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 128)       0

In [ ]:
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(train_data, batch_size=16, epochs=6, validation_data=val_data, steps_per_epoch=303//16, validation_steps=75//16)

Epoch 1/6
18/18 [==============================] - 169s 8s/step - loss: 3.3342 - accuracy: 0.3983 - val_loss: 1.1963 - val_accuracy: 0.3750
Epoch 2/6
18/18 [==============================] - 2s 100ms/step - loss: 0.7263 - accuracy: 0.6759 - val_loss: 1.3690 - val_accuracy: 0.0938
Epoch 3/6
18/18 [==============================] - 2s 99ms/step - loss: 0.4180 - accuracy: 0.8436 - val_loss: 2.3667 - val_accuracy: 0.3125
Epoch 4/6
18/18 [==============================] - 2s 98ms/step - loss: 0.2339 - accuracy: 0.9536 - val_loss: 2.3741 - val_accuracy: 0.0312
Epoch 5/6
18/18 [==============================] - 2s 97ms/step - loss: 0.0985 - accuracy: 0.9588 - val_loss: 1.5344 - val_accuracy: 0.3438
Epoch 6/6
18/18 [==============================] - 2s 97ms/step - loss: 0.1056 - accuracy: 0.9423 - val_loss: 2.7704 - val_accuracy: 0.7031


In [ ]:
save_model(filepath="/content/drive/My Drive/yüz tanıma/model1.h5",model=model)